In [1]:
import psycopg2

import ujson
import copy
import numpy as np
import scipy.stats
import sys

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

In [3]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')


time: 33.1 ms


In [4]:
dbname = 'endomondo'
conn = psycopg2.connect("dbname=endomondo user=patrickmulrooney")

# Open a cursor to perform database operations
cur = conn.cursor()


time: 7.51 ms


In [6]:
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
series_tables = []
workout_tables = []
for _i in cur.fetchall():
    if 'by_workout' in _i[0]:
        workout_tables.append(_i[0])
        continue
        
    series_tables.append(_i[0])

time: 18.9 ms


In [7]:
series_tables

['volleyball',
 'soccer',
 'run',
 'beach_volleyball',
 'martial_arts',
 'bike',
 'orienteering',
 'kite_surfing',
 'roller_skiing',
 'mountain_bike',
 'rugby',
 'pilates',
 'rowing',
 'sailing',
 'scuba_diving',
 'skate',
 'snowshoeing',
 'hiking',
 'skateboarding',
 'basketball',
 'cricket',
 'hockey',
 'american_football',
 'baseball',
 'boxing',
 'handball',
 'surfing',
 'user_workout_counts',
 'cross_country_skiing',
 'horseback_riding',
 'windsurfing',
 'yoga',
 'polo',
 'swimming',
 'step_counter',
 'table_tennis',
 'tennis',
 'treadmill_running',
 'kayaking',
 'fencing',
 'treadmill_walking',
 'walk_transport',
 'walk',
 'weight_training',
 'snowboarding',
 'aerobics',
 'golf',
 'gymnastics',
 'indoor_cycling',
 'wheelchair',
 'core_stability_training',
 'squash',
 'stair_climing',
 'dancing',
 'bike_transport',
 'circuit_training',
 'downhill_skiing',
 'climbing',
 'badminton',
 'elliptical',
 'fitness_walking',
 'weight_lifting']

time: 7.63 ms


In [8]:
workout_tables

['american_football_by_workout',
 'core_stability_training_by_workout',
 'baseball_by_workout',
 'handball_by_workout',
 'surfing_by_workout',
 'golf_by_workout',
 'circuit_training_by_workout',
 'horseback_riding_by_workout',
 'pilates_by_workout',
 'skateboarding_by_workout',
 'orienteering_by_workout',
 'sailing_by_workout',
 'dancing_by_workout',
 'mountain_bike_by_workout',
 'scuba_diving_by_workout',
 'fitness_walking_by_workout',
 'snowboarding_by_workout',
 'beach_volleyball_by_workout',
 'kite_surfing_by_workout',
 'kayaking_by_workout',
 'basketball_by_workout',
 'aerobics_by_workout',
 'rowing_by_workout',
 'rugby_by_workout',
 'swimming_by_workout',
 'treadmill_running_by_workout',
 'table_tennis_by_workout',
 'bike_transport_by_workout',
 'tennis_by_workout',
 'downhill_skiing_by_workout',
 'elliptical_by_workout',
 'cricket_by_workout',
 'windsurfing_by_workout',
 'yoga_by_workout',
 'badminton_by_workout',
 'roller_skiing_by_workout',
 'hiking_by_workout',
 'cross_countr

time: 5.73 ms


In [15]:
# Remove primary key if exists, and recreate

for _table in workout_tables + series_tables:
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS id;".format(_table)
    print query
    cur.execute(query)
    conn.commit()


ALTER TABLE snowboarding_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE fitness_walking_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE scuba_diving_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE aerobics_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE kayaking_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE basketball_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE kite_surfing_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE beach_volleyball_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE swimming_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE elliptical_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE table_tennis_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE treadmill_running_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE wheelchair_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE tennis_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE windsurfing_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE skate_by_workout DROP COLUMN IF EXISTS id;
ALTER TABLE yoga_by_workout 

In [16]:
# Create indexes
# CREATE UNIQUE INDEX title_idx ON films (title);
# Add id column,
# ALTER TABLE <TABLE> ADD COLUMN id SERIAL PRIMARY KEY;

for _table in workout_tables + series_tables:
    query = "ALTER TABLE {} ADD COLUMN id SERIAL PRIMARY KEY;".format(_table)
    print query
    cur.execute(query)
    conn.commit()


ALTER TABLE snowboarding_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE fitness_walking_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE scuba_diving_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE aerobics_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE kayaking_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE basketball_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE kite_surfing_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE beach_volleyball_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE swimming_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE elliptical_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE table_tennis_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE treadmill_running_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE wheelchair_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE tennis_by_workout ADD COLUMN id SERIAL PRIMARY KEY;
ALTER TABLE windsurfing_by_worko

In [ ]:
# Create indexes
# CREATE UNIQUE INDEX title_idx ON films (title);
workout_columns = ["gender", "workoutid", "userid", "start_time", "duration", "start_altitude", "start_latitude", "start_longitude", "series_length"]
for _table in workout_tables:
    for _col in workout_columns:
        query = "CREATE INDEX {}_{}_idx ON {} ({});".format(_table, _col, _table, _col)
        print _table, _col
        cur.execute(query)
        conn.commit()


series_columns = ["time", "altitude", "heart_rate", "latitude", "longitude", "speed", "workoutid"]
for _table in series_tables:
    for _col in series_columns:
        query = "CREATE INDEX {}_{}_idx ON {} ({});".format(_table, _col, _table, _col)
        print _table, _col
        cur.execute(query)
        conn.commit()


In [ ]:
# # Add id column,
# # alter table {} add column series_time_delta[];

# for _table in workout_tables:
#     query = "alter table {} add column series_time_delta integer[];".format(_table)
#     print query
#     cur.execute(query)
#     conn.commit()


In [ ]:
# update {}_by_workout set series_time_delta = (select array_agg(difference) from 
# (select 1 as dummy, foo.difference from (select time - lag(time) over 
# (order by time) as difference from {} where workoutid = {}) as foo 
# where difference is not null order by difference) as bar group by dummy) where workoutId = {};

for _table in sorted(series_tables):
    print _table
    query = "select workoutId from {}_by_workout".format(_table)
    cur.execute(query)
    workoutIds = [_i[0] for _i in cur.fetchall()]
    for _workout in workoutIds:
        #query = "update {}_by_workout set series_time_delta = (select array_agg(difference) from (select 1 as dummy, foo.difference from (select time - lag(time) over (order by time) as difference from {} where workoutid = {} order by time) as foo where difference is not null order by difference) as bar group by dummy) where workoutId = {};".format(_table, _table, _workout, _workout)
        query = "update {}_by_workout set series_time_delta = (select array_agg(difference) from (select 1 as dummy, foo.difference from (select time - lag(time) over (order by time) as difference from {} where workoutid = {} order by time) as foo where difference is not null) as bar group by dummy) where workoutId = {};".format(_table, _table, _workout, _workout)

        cur.execute(query)
        conn.commit()


aerobics
american_football
badminton
baseball
basketball
beach_volleyball
bike


In [18]:
# # Add id column,
# # alter table {} add column series_time_delta[];

# for _table in workout_tables:
#     query = "alter table {} add column series_time_delta_average numeric(20,10);".format(_table)
#     print query
#     cur.execute(query)
#     conn.commit()


time: 1.72 ms


In [19]:
#select workoutId, array_avg(series_time_delta) from bike_by_workout;
for _table in sorted(workout_tables):
    print _table
    query = "update {} set series_time_delta_average = array_avg(series_time_delta);".format(_table)
    cur.execute(query)
    conn.commit()
    

aerobics_by_workout
american_football_by_workout
badminton_by_workout
baseball_by_workout
basketball_by_workout
beach_volleyball_by_workout
bike_by_workout
bike_transport_by_workout
boxing_by_workout
circuit_training_by_workout
climbing_by_workout
core_stability_training_by_workout
cricket_by_workout
cross_country_skiing_by_workout
dancing_by_workout
downhill_skiing_by_workout
elliptical_by_workout
fencing_by_workout
fitness_walking_by_workout
golf_by_workout
gymnastics_by_workout
handball_by_workout
hiking_by_workout
hockey_by_workout
horseback_riding_by_workout
indoor_cycling_by_workout
kayaking_by_workout
kite_surfing_by_workout
martial_arts_by_workout
mountain_bike_by_workout
orienteering_by_workout
pilates_by_workout
polo_by_workout
roller_skiing_by_workout
rowing_by_workout
rugby_by_workout
run_by_workout
sailing_by_workout
scuba_diving_by_workout
skate_by_workout
skateboarding_by_workout
snowboarding_by_workout
snowshoeing_by_workout
soccer_by_workout
squash_by_workout
stair_cli

In [27]:
users = []

for _table in sorted(series_tables):
    query = "select userid from {}_by_workout".format(_table)
    cur.execute(query)
    users += [_i[0] for _i in cur.fetchall()]

# Unique
users = set(users)

time: 1.02 s


In [32]:
workout_fields = str.join(",",[ _i + ' integer' for _i in series_tables])

query = "create table user_workout_counts (id SERIAL PRIMARY KEY, userid integer, {} );".format(workout_fields)
print query
cur.execute(query)
conn.commit()

create table user_workout_counts (id SERIAL PRIMARY KEY, userid integer, volleyball integer,soccer integer,run integer,beach_volleyball integer,bike integer,kite_surfing integer,martial_arts integer,mountain_bike integer,orienteering integer,pilates integer,roller_skiing integer,rowing integer,rugby integer,sailing integer,scuba_diving integer,skate integer,snowshoeing integer,hiking integer,skateboarding integer,basketball integer,cricket integer,hockey integer,american_football integer,baseball integer,boxing integer,handball integer,surfing integer,cross_country_skiing integer,horseback_riding integer,windsurfing integer,yoga integer,polo integer,swimming integer,step_counter integer,table_tennis integer,tennis integer,treadmill_running integer,kayaking integer,fencing integer,treadmill_walking integer,walk_transport integer,walk integer,weight_training integer,snowboarding integer,aerobics integer,golf integer,gymnastics integer,indoor_cycling integer,wheelchair integer,squash inte

In [53]:
for _user in users:
    workout_counts = []
    for _workout in series_tables:
        query = "select count(*) from {}_by_workout where userid = {}".format(_workout, _user)
        cur.execute(query)
        workout_counts.append(str(int(cur.fetchone()[0])))
    
    query = "insert into user_workout_counts values (DEFAULT, {}, {});".format(_user, str.join(",", workout_counts))
    cur.execute(query)
    conn.commit()

time: 10min 36s


In [56]:
import datetime
import pytz

from timezonefinder import TimezoneFinder

TypeError: u'b1(i4,i4,i4[:,:])' is not a callable object

time: 31.8 ms


In [ ]:
tf = TimezoneFinder()

# TimezoneFinder.using_numba() 

In [ ]:
# # Add id column,
# # alter table {} add column series_time_delta[];

# for _table in workout_tables:
#     query = "alter table {} add column timezone varchar;".format(_table)
#     cur.execute(query)
#     conn.commit()


In [ ]:
# add timezone
for _table in workout_tables:
    print _table
    query = "select start_latitude, start_longitude, workoutId from {}".format(_table)
    cur.execute(query)
    workoutIds = [[_i[2], tf.timezone_at(lat=_i[0], lng=_i[1])] for _i in cur.fetchall()]

    for _workout in workoutIds:
        if _workout[1] == None:
            continue
        query = "update {} set timezone = '{}' where workoutId = {};".format(_table, _workout[1], _workout[0])
        
        cur.execute(query)
        conn.commit()


In [ ]:
# update run set altitude = NULL, latitude = NULL, longitude = NULL where altitude = -500.0 and latitude = 0.0 and longitude =
0.0;

In [ ]:
#select workoutId, array_avg(series_time_delta) from bike_by_workout;
workout = []
for _table in workout_tables:
    query = "select series_length from {};".format(_table)
    cur.execute(query)
    workout.append([_table, [_i[0] for _i in cur.fetchall()]])

workout = sorted(workout, key=lambda x: len(x[1]), reverse=True)

In [ ]:
for _workout in workout:
    print _workout[0], len(_workout[1])
    plt.hist(_workout[1])
    plt.show()